In [1]:
from IPython import display
import os

# function to run dash codes from jupyternotebook 
def show_app(app, port = 9999, 
             width = 700, 
             height = 350, 
             offline = False,
            in_binder = None):
    in_binder ='JUPYTERHUB_SERVICE_PREFIX' in os.environ if in_binder is None else in_binder
    if in_binder:
        base_prefix = '{}proxy/{}/'.format(os.environ['JUPYTERHUB_SERVICE_PREFIX'], port)
        url = 'https://hub.mybinder.org{}'.format(base_prefix)
        app.config.requests_pathname_prefix = base_prefix
    else:
        url = 'http://localhost:%d' % port
        
    iframe = '<a href="{url}" target="_new">Open in new window</a><hr><iframe src="{url}" width={width} height={height}></iframe>'.format(url = url, 
                                                                                  width = width, 
                                                                                  height = height)
    
    display.display_html(iframe, raw = True)
    if offline:
        app.css.config.serve_locally = True
        app.scripts.config.serve_locally = True
    return app.run_server(debug=False, # needs to be false in Jupyter
                          host = '0.0.0.0',
                          port=port)

In [2]:
%run purchase_stock.ipynb

In [1]:
import robin_stocks as rs
import numpy as np
from datetime import datetime

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objects as go


username = ''
password = ''

# log in your robinhood account 
login = rs.login(username, password)

# monitor time frame
min_time = datetime(datetime.today().year,
                    datetime.today().month,
                    datetime.today().day,
                    0, 0, 0)

max_time = datetime(datetime.today().year,
                    datetime.today().month,
                    datetime.today().day,
                    21, 0, 0)

p_list = []
t_list = []


#return a list of stocks
stocks_list = get_algo_stock_list()

# return latest buying power in your account
def get_latest_buyPower():
    stocks_list = get_algo_stock_list()
    principle = float(rs.profiles.load_account_profile()['cash_available_for_withdrawal'])
    principle -= 25000
    #stock_price_list = rs.stocks.get_latest_price(stocks_list)
    print(stocks_list)
    if stocks_list:
        stock_price_list = rs.stocks.get_latest_price(inputSymbols=stocks_list)
        
    
        for stock in stock_price_list:
            principle += float(stock)

    return principle, datetime.now()


# Initiating a dash app
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# simple app interface
app.layout = html.Div(children=[
    html.Div([
        html.H5('Live Graph Testing'),
    ], className='row'),
    html.Div([
        dcc.Graph(id='live_graph', animate=True),
        dcc.Interval(id='interval_component', interval=20*1000, n_intervals=0)  # interval = 20 sec
    ])
])

# update live chart
@app.callback(Output('live_graph', 'figure'),
              [Input('interval_component', 'n_intervals')])
def update_graph(n):
    global stocks_list
    global min_time
    global max_time
    global p_list
    global t_list
    
    # define x-axis
    min_time = datetime.now()
    min_time = min_time.replace(minute=30, hour=6, second=0, microsecond=0)
    
    max_time = datetime.now()
    max_time = max_time.replace(minute=0, hour=13, second=0, microsecond=0)
    
    # get buy power at that time instance
    principle, current_time = get_latest_buyPower()

    p_list.append(principle)
    t_list.append(current_time)
    
    # dash plot data 
    trace = go.Scatter(x=t_list,
                       y=p_list,
                       name='random plot',
                       mode='lines+markers')

    layout = go.Layout(xaxis=dict(range=[min_time, max_time]), yaxis=dict(range=[0, principle + 10]))
    fig = {'data': [trace],
           'layout': layout
           }

    return fig


if __name__ == '__main__':
    app.run_server(debug=False)


KeyboardInterrupt: 